In [52]:
# DO NOT modify this cell. 
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [219]:
filename = "CAS.pgm"
verticalSeam2Remove = 20
horizontalSeam2Remove = 50

In [239]:
import numpy as np


In [240]:
# Part I : Vertical seam removal only
# Save your processed file to img_processed_v_h.pgm
filename1 = filename.split(".")[0]+"_M_processed_"+str(verticalSeam2Remove)+"_0.pgm"
print(filename1)

# function to read the input image
def parse_pgm(filename):
    with open(filename, 'r') as file:
        # Read and discard the header and comment lines
        magic = file.readline()
        comment = file.readline()

        # Read width and height
        width, height = map(int, file.readline().split())

        # Read and discard the max value indicator
        max_color_value = file.readline()

        # Read pixel values into a flat list
        pixel_values = list(map(int, file.read().split()))

    # Reshape the flat list into a 2D array
    image = [pixel_values[i * width:(i + 1) * width] for i in range(height)]

    return image


# function to create an output file
def output_file(out_file, out_file_name):
    with open(out_file_name, 'w') as file:
        # header
        file.write('P2\n')
        file.write('This is the output image\n')
        file.write(f'{len(out_file[0])} {len(out_file)}\n')
        file.write('255\n')

        # pixel body
        for row in out_file:
            for column in row:
                file.write(str(column))
                file.write(" ")
            file.write("\n")

def main():
    # input the image to seam
    img = parse_pgm(filename)
    
    # Output the image as text
    output_file(img, filename1)


if __name__ == '__main__':
    main()


CAS_M_processed_20_0.pgm


In [3]:
# Part II : both vertical and horizontal seams removal 
# Save your processed file to img_processed_v_h.pgm
filename2 = filename.split(".")[0]+"_processed_"+ str(verticalSeam2Remove)+"_"+str(horizontalSeam2Remove)+".pgm"
print(filename2)

# your code, add cells as you need

Buchtel_processed_10_8.pgm
